# Teste pyomo

In [25]:
import numpy as np
from udgp import Instance

In [26]:
instance = Instance.artificial_molecule(5, freq=True, seed=123456)
instance.view_input()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [27]:
import pyomo.environ as pyo

In [28]:
instance.fixed_n

1

In [37]:
model = pyo.ConcreteModel()

model.n = pyo.Param(initialize=instance.n)
model.m = pyo.Param(initialize=instance.m)

## CONJUNTOS
ny = 1
rng = np.random.default_rng()
y_indices = rng.choice(instance.fixed_n, ny, replace=False)
model.Iy = pyo.Set(initialize=y_indices)

nx = 4
x_indices = np.arange(instance.fixed_n, nx + instance.fixed_n)
model.Ix = pyo.Set(initialize=x_indices)

model.I = model.Iy | model.Ix

k = np.arange(instance.m)[instance.freqs != 0]
model.K = pyo.Set(initialize=k)

model.IJ = pyo.Set(
    within=model.I * model.I,
    initialize=((i, j) for i in model.I for j in model.I if i < j and j not in model.Iy),
)

print(model.IJ.data())


((0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4))


In [38]:
model.L = pyo.RangeSet(3)

## PARÂMETROS
model.d_max = pyo.Param(initialize=instance.dists.max())
model.dists = pyo.Param(
    model.K, 
    within=pyo.PositiveReals, 
    initialize=instance.dists
)
model.freqs = pyo.Param(
    model.K, 
    within=pyo.NonNegativeIntegers, 
    initialize=instance.freqs,
)

## VARIÁVEIS BASE
model.a = pyo.Var(model.IJ, model.K, within=pyo.Binary)
model.x = pyo.Var(model.I, model.L, within=pyo.Reals)
model.v = pyo.Var(model.IJ, model.L, within=pyo.Reals)
model.r = pyo.Var(model.IJ, within=pyo.NonNegativeReals)

## VARIÁVEIS MACULAN
model.p = pyo.Var(model.IJ, model.K, within=pyo.Reals)
model.w = pyo.Var(model.IJ, model.K, within=pyo.NonNegativeReals)
model.z = pyo.Var(model.IJ, model.K, within=pyo.NonNegativeReals)


## RESTRIÇÕES BASE
@model.Constraint(model.K)
def constr_a1(model, k):
    return sum(model.a[i, j, k] for i, j in model.IJ) <= model.freqs[k]


@model.Constraint(model.IJ)
def constr_a2(model, i, j):
    return sum(model.a[i, j, k] for k in model.K) == 1


@model.Constraint(model.IJ, model.L)
def constr_v(model, i, j, l):
    return model.v[i, j, l] == model.x[j, l] - model.x[i, l]


@model.Constraint(model.IJ)
def constr_r(model, i, j):
    return model.r[i, j] ** 2 == sum(model.v[i, j, l] ** 2 for l in model.L)


## RESTRIÇÕES MACULAN
@model.Constraint(model.IJ, model.K)
def constr_x1(model, i, j, k):
    return model.w[i, j, k] >= model.p[i, j, k]


@model.Constraint(model.IJ, model.K)
def constr_x2(model, i, j, k):
    return model.w[i, j, k] >= -model.p[i, j, k]


@model.Constraint(model.IJ, model.K)
def constr_x3(model, i, j, k):
    return model.z[i, j, k] >= model.r[i, j] - model.d_max * (1 - model.a[i, j, k])


@model.Constraint(model.IJ, model.K)
def constr_x4(model, i, j, k):
    return model.z[i, j, k] <= model.r[i, j] + model.d_max * (1 - model.a[i, j, k])


@model.Constraint(model.IJ, model.K)
def constr_x5(model, i, j, k):
    return model.z[i, j, k] >= -model.d_max * model.a[i, j, k]


@model.Constraint(model.IJ, model.K)
def constr_x6(model, i, j, k):
    return model.z[i, j, k] <= model.d_max * model.a[i, j, k]


@model.Constraint(model.IJ, model.K)
def constr_x7(model, i, j, k):
    return model.z[i, j, k] >= model.dists[k] + model.p[i, j, k] - model.d_max * (1 - model.a[i, j, k])


@model.Constraint(model.IJ, model.K)
def constr_x8(model, i, j, k):
    return model.z[i, j, k] <= model.dists[k] + model.p[i, j, k] + model.d_max * (1 - model.a[i, j, k])


## OBJETIVO
@model.Objective(sense=pyo.minimize)
def objective(model):
    return 1 + pyo.summation(model.w)

In [40]:
opt = pyo.SolverFactory('gurobi')

opt.options['NonConvex'] = 2
opt.options['MIPGap'] = 1e-1
opt.options['IntFeasTol'] = 1e-4
opt.options['FeasibilityTol'] = 1e-4
opt.options['OptimalityTol'] = 1e-4

opt.solve(model, tee=True, report_timing=True)


        0.01 seconds required to write file
        0.01 seconds required for presolve


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2372514
Academic license - for non-commercial use only - registered to braun@cos.ufrj.br
Read LP format model from file C:\Users\Usuario\AppData\Local\Temp\tmp4bv83gfy.pyomo.lp
Reading time = 0.00 seconds
x1: 445 rows, 256 columns, 1190 nonzeros
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0.1
Set parameter IntFeasTol to value 0.0001
Set parameter FeasibilityTol to value 0.0001
Set parameter OptimalityTol to value 0.0001
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license - for non-commercial use only - registered to braun@cos.ufrj.br
Optimize a model with 445 rows, 256 columns and 1190 nonzeros
Model fingerprint: 0xb11a873c
Model has 10 quadratic constraints
Variable types: 206 continuous, 50 intege

KeyboardInterrupt: 

In [18]:
points = np.array([[model.x[i, l].value for l in model.L] for i in model.I])
instance.points = points

In [19]:
instance.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol